<a href="https://colab.research.google.com/github/satyamverma95/DL_NLP/blob/main/Pre_requisite_finder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

If you're opening this Notebook on colab, you will probably need to install 🤗 Transformers and 🤗 Datasets. Uncomment the following cell and run it.

In [4]:
#!pip install evaluate
#!pip install huggingface_hub
#!pip install datasets
#!pip install transformers==4.28.0
#!pip install --upgrade accelerate

Getting Token authentication cell and input your username and password:

In [5]:
from huggingface_hub import notebook_login

notebook_login()

In [6]:
import transformers

print(transformers.__version__)


4.28.0


# Fine-tuning a model on a text classification task

This notebook is built to run on any of the tasks in the list above, with any model checkpoint from the [Model Hub](https://huggingface.co/models) as long as that model has a version with a classification head. Depending on you model and the GPU you are using, you might need to adjust the batch size to avoid out-of-memory errors. Set those three parameters, then the rest of the notebook should run smoothly:

In [7]:
from datasets import  load_metric

task = "rte"
model_checkpoint = "distilbert-base-uncased"
batch_size = 16
metric = load_metric('glue', task)

<ipython-input-7-6da0497477ac>:6: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric('glue', task)


The DistilBERT model has several pre-trained variants with different configurations. As of my knowledge cutoff in September 2021, the following DistilBERT models were available:

1. `distilbert-base-uncased`: The base variant of DistilBERT with an uncased vocabulary.
2. `distilbert-base-uncased-distilled-squad`: A variant of DistilBERT fine-tuned on the Stanford Question Answering Dataset (SQuAD). It is specifically trained for question answering tasks.
3. `distilbert-base-cased`: Similar to `distilbert-base-uncased`, but with a cased vocabulary.
4. `distilbert-base-cased-distilled-squad`: Similar to `distilbert-base-uncased-distilled-squad`, but with a cased vocabulary.
5. `distilbert-base-multilingual-cased`: A variant of DistilBERT trained on a multilingual corpus with a cased vocabulary. It supports multiple languages.
6. `distilbert-base-german-cased`: A variant of DistilBERT trained specifically for the German language with a cased vocabulary.

Please note that new variants of DistilBERT may have been introduced since my knowledge cutoff. To get the most up-to-date information on available DistilBERT models, I recommend referring to the Hugging Face Model Hub or the official DistilBERT repository.

## Loading the dataset

Mounting the Google drive Content Folder.
Uploading the file to Content Folder.

In [8]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

Mounted at /content/drive/


In [10]:
csv_file_path = '/content/Pre-requisite_dataset#2.csv'

In [11]:
from datasets import DatasetDict, Dataset, load_dataset, load_metric
dataset = Dataset.from_csv(csv_file_path)

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-864c48e558695b6c/0.0.0. Subsequent calls will reuse this data.


In [ ]:
#def convert_label(example):
#    example['label'] = 1 if example['label'] == 'Yes' else 0
#    return example

In [ ]:
#dataset = dataset.map(convert_label)

To access an actual element, using a random index:

In [12]:
dataset[4]

{'Topic': 'Learning image features using 2d convolutional DBNs',
 'Prerequsites': 'Applications of deep networks',
 'label': 1}

Splitting the Data in Train and Test with 80% and 20% ratio respectively. 

In [13]:
dataset = dataset.train_test_split(test_size=0.2)

The `dataset` object itself is [`DatasetDict`](https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasetdict), which contains one key for the training, validation and test set.

Showing the overall structure of the Dataset dictonary.

In [14]:
dataset

DatasetDict({
    train: Dataset({
        features: ['Topic', 'Prerequsites', 'label'],
        num_rows: 10368
    })
    test: Dataset({
        features: ['Topic', 'Prerequsites', 'label'],
        num_rows: 2592
    })
})

To get a sense of what the data looks like, the following function will show some examples picked randomly in the dataset.

In [15]:
import datasets
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [17]:
show_random_elements(dataset["train"])

,Topic,Prerequsites,label
0,Learning from complete data,Directed graphical models (Bayes nets),0
1,Using the cdf,Expectation propagation,0
2,Variational inference,Exact inference for graphical models,0
3,Restricted Boltzmann machines (RBMs),Fitting using batch variational inference,0
4,Digression: The Wishart distribution *,Linear discriminant analysis (LDA),0
5,Non-convex regularizers,Group Lasso,1
6,Deep multi-layer perceptrons,Supervised LDA,0
7,Likelihood,"Example: change of variables, the MC way",1
8,Ensemble learning,Other kinds of neural networks,1
9,Gaussian (normal) distribution,A brief review of probability theory,0


## Preprocessing the data

Before we can feed those texts to our model, we need to preprocess them. This is done by a 🤗 Transformers `Tokenizer` which will (as the name indicates) tokenize the inputs (including converting the tokens to their corresponding IDs in the pretrained vocabulary) and put it in a format the model expects, as well as generate the other inputs that model requires.

To do all of this, we instantiate our tokenizer with the `AutoTokenizer.from_pretrained` method, which will ensure:

- we get a tokenizer that corresponds to the model architecture we want to use,
- we download the vocabulary used when pretraining this specific checkpoint.

That vocabulary will be cached, so it's not downloaded again the next time we run the cell.

In [18]:
from transformers import AutoTokenizer
from transformers import BertTokenizer    

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased", use_fast=True)
#tokenizer = BertTokenizer.from_pretrained("bert-base-uncased", use_fast=True)

We pass along `use_fast=True` to the call above to use one of the fast tokenizers (backed by Rust) from the 🤗 Tokenizers library. Those fast tokenizers are available for almost all models, but if you got an error with the previous call, remove that argument.

In [19]:
task_to_keys = {
                "mrpc": ("Topic", "Prerequsites"),
                "rte": ("Topic", "Prerequsites"),
              }

We can double check it does work on our current dataset:

In [20]:
sentence1_key, sentence2_key = task_to_keys[task]
print("sentence1_key :{}, sentence2_key:{}".format(sentence1_key, sentence2_key))
if sentence2_key is None:
    print(f"Sentence: {dataset['train'][0][sentence1_key]}")
else:
    print(f"Topic: {dataset['train'][0][sentence1_key]}")
    print(f"Prerequsites: {dataset['train'][0][sentence2_key]}")

sentence1_key :Topic, sentence2_key:Prerequsites
Topic: Kernels for comparing documents
Prerequsites: Bridge regression


We can then write the function that will preprocess our samples. We just feed them to the `tokenizer` with the argument `truncation=True`. This will ensure that an input longer that what the model selected can handle will be truncated to the maximum length accepted by the model.

In [21]:
def preprocess_function(examples):
    return tokenizer(examples[sentence1_key], examples[sentence2_key], truncation='do_not_truncate', max_length=100, padding="max_length")

This function works with one or several examples. In the case of several examples, the tokenizer will return a list of lists for each key:

In [22]:
preprocess_function(dataset['train'][:2])

{'input_ids': [[101, 16293, 2015, 2005, 13599, 5491, 102, 2958, 26237, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [101, 19143, 102, 6053, 2361, 2006, 3940, 14244, 4275, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 

To apply this function on all the sentences (or pairs of sentences) in our dataset, we just use the `map` method of our `dataset` object we created earlier. This will apply the function on all the elements of all the splits in `dataset`, so our training, validation and testing data will be preprocessed in one single command.

In [23]:
encoded_dataset = dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/10368 [00:00<?, ? examples/s]

Map:   0%|          | 0/2592 [00:00<?, ? examples/s]

## Fine-tuning the model

Now that our data is ready, we can download the pretrained model and fine-tune it. Since all our tasks are about sentence classification, we use the `AutoModelForSequenceClassification` class. Like with the tokenizer, the `from_pretrained` method will download and cache the model for us. The only thing we have to specify is the number of labels for our problem.:

In [24]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
from transformers import BertTokenizer, BertForSequenceClassification

num_labels = 2
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels)
#model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=num_labels)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_projector.bias', 'vocab_projector.weight', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'classifier.bias', 'classifier

To instantiate a `Trainer`, we will need to define two more things. The most important is the [`TrainingArguments`](https://huggingface.co/transformers/main_classes/trainer.html#transformers.TrainingArguments), which is a class that contains all the attributes to customize the training. It requires one folder name, which will be used to save the checkpoints of the model, and all other arguments are optional:

In [27]:
metric_name = "pearson" if task == "stsb" else "matthews_correlation" if task == "cola" else "accuracy"
model_name = model_checkpoint.split("/")[-1]

args = TrainingArguments(
    f"{model_name}-finetuned-Pre_requisite_finder",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=2.2534703769467627e-05,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
    push_to_hub=True,
    seed = 37
)

Here we set the evaluation to be done at the end of each epoch, tweak the learning rate, use the `batch_size` defined at the top of the notebook and customize the number of epochs for training, as well as the weight decay. Since the best model might not be the one at the end of training, we ask the `Trainer` to load the best model it saved (according to `metric_name`) at the end of training.

The last argument to setup everything so we can push the model to the [Hub](https://huggingface.co/models) regularly during training. Remove it if you didn't follow the installation steps at the top of the notebook. If you want to save your model locally in a name that is different than the name of the repository it will be pushed, or if you want to push your model under an organization and not your name space, use the `hub_model_id` argument to set the repo name (it needs to be the full name, including your namespace: for instance `"sgugger/bert-finetuned-mrpc"` or `"huggingface/bert-finetuned-mrpc"`).

The last thing to define for our `Trainer` is how to compute the metrics from the predictions. We need to define a function for this, which will just use the `metric` we loaded earlier, the only preprocessing we have to do is to take the argmax of our predicted logits (our just squeeze the last axis in the case of STS-B):

In [28]:
def write_to_file(filename, data):
  with open(filename, 'w', encoding='utf-8') as file:
    file.write(data)

In [29]:
import pandas as pd
pd.set_option('display.max_rows', None)

In [30]:
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    #print("Eval", eval_pred)
    #print("predictions : {}, labels : {}".format(predictions, labels))
    predictions = np.argmax(predictions, axis=1)
    accuracy = metric.compute(predictions=predictions, references=labels)
    print ("predictions :{}, accuracy : {}".format(str(predictions), accuracy))
    #write_to_file("/content/Predictions.csv", str(predictions.tolist()))
    return (accuracy)

Then we just need to pass all of this along with our datasets to the `Trainer`:

In [31]:
from transformers import DefaultDataCollator

data_collator = DefaultDataCollator()

In [32]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

Cloning https://huggingface.co/satyamverma/distilbert-base-uncased-finetuned-Pre_requisite_finder into local empty directory.


Download file pytorch_model.bin:   0%|          | 3.40k/255M [00:00<?, ?B/s]

Download file training_args.bin: 100%|##########| 3.56k/3.56k [00:00<?, ?B/s]

Download file runs/May23_10-18-20_fcd100884581/1684837128.6869931/events.out.tfevents.1684837128.fcd100884581.…

Download file runs/May23_10-18-20_fcd100884581/events.out.tfevents.1684837128.fcd100884581.908.0: 100%|#######…

Clean file training_args.bin:  28%|##8       | 1.00k/3.56k [00:00<?, ?B/s]

Clean file runs/May23_10-18-20_fcd100884581/1684837128.6869931/events.out.tfevents.1684837128.fcd100884581.908…

Clean file runs/May23_10-18-20_fcd100884581/events.out.tfevents.1684837128.fcd100884581.908.0:  16%|#5        …

Clean file pytorch_model.bin:   0%|          | 1.00k/255M [00:00<?, ?B/s]

You might wonder why we pass along the `tokenizer` when we already preprocessed our data. This is because we will use it once last time to make all the samples we gather the same length by applying padding, which requires knowing the model's preferences regarding padding (to the left or right? with which token?). The `tokenizer` has a pad method that will do all of this right for us, and the `Trainer` will use it. You can customize this part by defining and passing your own `data_collator` which will receive the samples like the dictionaries seen above and will need to return a dictionary of tensors.

We can now finetune our model by just calling the `train` method:

In [33]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
1,0.694500,0.678808,0.544753
2,0.680500,0.675450,0.546682
3,0.665000,0.675313,0.543210


predictions :[1 0 0 ... 0 0 0], accuracy : {'accuracy': 0.5447530864197531}
predictions :[1 0 0 ... 0 0 0], accuracy : {'accuracy': 0.5466820987654321}
predictions :[1 0 0 ... 0 0 0], accuracy : {'accuracy': 0.5432098765432098}


TrainOutput(global_step=1944, training_loss=0.6766352084438496, metrics={'train_runtime': 385.9597, 'train_samples_per_second': 80.589, 'train_steps_per_second': 5.037, 'total_flos': 804739446835200.0, 'train_loss': 0.6766352084438496, 'epoch': 3.0})

We can check with the `evaluate` method that our `Trainer` did reload the best model properly (if it was not the last one):

In [34]:
trainer.evaluate()

predictions :[1 0 0 ... 0 0 0], accuracy : {'accuracy': 0.5466820987654321}


{'eval_loss': 0.6754501461982727,
 'eval_accuracy': 0.5466820987654321,
 'eval_runtime': 7.8387,
 'eval_samples_per_second': 330.668,
 'eval_steps_per_second': 20.667,
 'epoch': 3.0}

You can now upload the result of the training to the Hub, just execute this instruction:

In [35]:
trainer.push_to_hub()

Upload file runs/May23_14-24-44_73c6d40f3ea2/events.out.tfevents.1684851988.73c6d40f3ea2.205.0:   0%|         …

Upload file runs/May23_14-24-44_73c6d40f3ea2/events.out.tfevents.1684852381.73c6d40f3ea2.205.2:   0%|         …

To https://huggingface.co/satyamverma/distilbert-base-uncased-finetuned-Pre_requisite_finder
   0fd5be8..49208d8  main -> main

   0fd5be8..49208d8  main -> main

To https://huggingface.co/satyamverma/distilbert-base-uncased-finetuned-Pre_requisite_finder
   49208d8..cf5bb7f  main -> main

   49208d8..cf5bb7f  main -> main



'https://huggingface.co/satyamverma/distilbert-base-uncased-finetuned-Pre_requisite_finder/commit/49208d8ab84b852f702a55c5cd87bab6b01e9017'

## Hyperparameter search

The `Trainer` supports hyperparameter search using [optuna](https://optuna.org/) or [Ray Tune](https://docs.ray.io/en/latest/tune/). For this last section you will need either of those libraries installed, just uncomment the line you want on the next cell and run it.

In [38]:
#! pip install optuna
#! pip install ray[tune]

During hyperparameter search, the `Trainer` will run several trainings, so it needs to have the model defined via a function (so it can be reinitialized at each new run) instead of just having it passed. We jsut use the same function as before:

In [39]:
def model_init():
    return AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels)

And we can instantiate our `Trainer` like before:

In [41]:
trainer = Trainer(
    model_init=model_init,
    args=args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_projector.bias', 'vocab_projector.weight', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'classifier.bias', 'classifier

The method we call this time is `hyperparameter_search`. Note that it can take a long time to run on the full dataset for some of the tasks. You can try to find some good hyperparameter on a portion of the training dataset by replacing the `train_dataset` line above by:
```python
train_dataset = encoded_dataset["train"].shard(index=1, num_shards=10) 
```
for 1/10th of the dataset. Then you can run a full training on the best hyperparameters picked by the search.

In [42]:
best_run = trainer.hyperparameter_search(n_trials=10, direction="maximize")

[I 2023-05-23 14:44:44,344] A new study created in memory with name: no-name-4337075c-cb95-41a8-b977-62b510b09d08
Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_projector.bias', 'vocab_projector.weight', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpo

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.694793,0.487269


predictions :[0 0 0 ... 0 0 0], accuracy : {'accuracy': 0.48726851851851855}


[I 2023-05-23 14:46:35,513] Trial 0 finished with value: 0.48726851851851855 and parameters: {'learning_rate': 5.573465112166566e-06, 'num_train_epochs': 1, 'seed': 17, 'per_device_train_batch_size': 64}. Best is trial 0 with value: 0.48726851851851855.
Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_projector.bias', 'vocab_projector.weight', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model 

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.693485,0.499228
2,No log,0.676445,0.538194
3,No log,0.672548,0.542438
4,0.684100,0.673563,0.543596


...

[I 2023-05-23 14:53:59,232] Trial 1 finished with value: 0.5435956790123457 and parameters: {'learning_rate': 1.5984748475703943e-05, 'num_train_epochs': 4, 'seed': 16, 'per_device_train_batch_size': 64}. Best is trial 1 with value: 0.5435956790123457.
Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_projector.bias', 'vocab_projector.weight', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model f

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.690546,0.486883
2,No log,0.682176,0.536265


predictions :[1 0 1 ... 1 1 1], accuracy : {'accuracy': 0.4868827160493827}
predictions :[1 0 0 ... 0 0 0], accuracy : {'accuracy': 0.5362654320987654}


[W 2023-05-23 14:57:35,031] Trial 2 failed with parameters: {'learning_rate': 1.234687409096097e-05, 'num_train_epochs': 2, 'seed': 39, 'per_device_train_batch_size': 64} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/optuna/study/_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
  File "/usr/local/lib/python3.10/dist-packages/transformers/integrations.py", line 198, in _objective
    trainer.train(resume_from_checkpoint=checkpoint, trial=trial)
  File "/usr/local/lib/python3.10/dist-packages/transformers/trainer.py", line 1662, in train
    return inner_training_loop(
  File "/usr/local/lib/python3.10/dist-packages/transformers/trainer.py", line 2021, in _inner_training_loop
    self._maybe_log_save_evaluate(tr_loss, model, trial, epoch, ignore_keys_for_eval)
  File "/usr/local/lib/python3.10/dist-packages/transformers/trainer.py", line 2291, in _maybe_log_save_evaluate
    

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 1>:1                                                                              │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/trainer.py:2592 in hyperparameter_search    │
│                                                                                                  │
│   2589 │   │   │   HPSearchBackend.SIGOPT: run_hp_search_sigopt,                                 │
│   2590 │   │   │   HPSearchBackend.WANDB: run_hp_search_wandb,                                   │
│   2591 │   │   }                                                                                 │
│ ❱ 2592 │   │   best_run = backend_dict[backend](self, n_trials, direction, **kwargs)             │
│   2593 │   │                                                                                     │
│   2594 │   │   self.hp_search_backend = None                                                     │
│   2595 │   │   return best_run                                                                   │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/integrations.py:208 in run_hp_search_optuna │
│                                                                                                  │
│    205 │   │   timeout = kwargs.pop("timeout", None)                                             │
│    206 │   │   n_jobs = kwargs.pop("n_jobs", 1)                                                  │
│    207 │   │   study = optuna.create_study(direction=direction, **kwargs)                        │
│ ❱  208 │   │   study.optimize(_objective, n_trials=n_trials, timeout=timeout, n_jobs=n_jobs)     │
│    209 │   │   best_trial = study.best_trial                                                     │
│    210 │   │   return BestRun(str(best_trial.number), best_trial.value, best_trial.params)       │
│    211 │   else:                                                                                 │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/optuna/study/study.py:425 in optimize                    │
│                                                                                                  │
│    422 │   │   │   │   If nested invocation of this method occurs.                               │
│    423 │   │   """                                                                               │
│    424 │   │                                                                                     │
│ ❱  425 │   │   _optimize(                                                                        │
│    426 │   │   │   study=self,                                                                   │
│    427 │   │   │   func=func,                                                                    │
│    428 │   │   │   n_trials=n_trials,                                                            │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/optuna/study/_optimize.py:66 in _optimize                │
│                                                                                                  │
│    63 │                                                                                          │
│    64 │   try:                                                                                   │
│    65 │   │   if n_jobs == 1:                                                                    │
│ ❱  66 │   │   │   _optimize_sequential(                                                          │
│    67 │   │   │   │   study,                               

The `hyperparameter_search` method returns a `BestRun` objects, which contains the value of the objective maximized (by default the sum of all metrics) and the hyperparameters it used for that run.

In [ ]:
best_run

You can customize the objective to maximize by passing along a `compute_objective` function to the `hyperparameter_search` method, and you can customize the search space by passing a `hp_space` argument to `hyperparameter_search`. See this [forum post](https://discuss.huggingface.co/t/using-hyperparameter-search-in-trainer/785/10) for some examples.

To reproduce the best training, just set the hyperparameters in your `TrainingArgument` before creating a `Trainer`:

In [ ]:
for n, v in best_run.hyperparameters.items():
    setattr(trainer.args, n, v)

trainer.train()

In [ ]:
trainer.push_to_hub()